In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

import os
import sys
from pprint import pprint as pp

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def flatten(arr, c, max_recurse):
    """Flattens nested array while tracking indices for each level.
    
    There must be poly_np of structure: 
        np.arange(12).reshape(4, 3)  # triangle 
    as final elements of the nested arr. 
    
    Internal variables:
        pi: Previous Indices. Keeps track of all previous 
            indices and concatenates them at each recursion.
        li: Local Index. An index that restarts iteration 
            for each sub-array in a given list. Functions as 
            local "counter", i.e. will tell you how many 
            polygons are in the prior level (sub-building)  
        fi: Flattened Index. An index that does not restart 
            iteration for each sub-array in a given list. 
            This functions as a global "counter", i.e 
            will tell you how many polygons are in the 
            whole building.

    Returns:
        Array of list of tracked indices for each level, 
        and a 4d point: 
            [[0, (1, 0), (1, 0), ptx, pty, ptz,  1], ... ] 
        Note the tuples indicates (li, fi) pairs for each level.
    """
    
    def _is_poly(arr):
        """Checks if first item is a homogenous point."""
        return isinstance(arr[0][1], np.ndarray) and \
               arr[0][1][:, 0].shape == (4,)

    if c >= max_recurse: 
        raise Exception("Recursion exceeded from specified amount "
            "in c={} >= max_recurse={}. Check flatten()."
            .format(c, max_recurse))
             
    if _is_poly(arr):
        fi = -1
        return [pi + [(li, fi:=fi+1)] + n.tolist() 
                for pi, m in arr 
                for li, n in enumerate(m.T)]  
    
    if c == 0:
        return flatten([([(pi,)], m) 
                        for pi, m in enumerate(arr)], 
                        c+1, max_recurse)    
    
    fi = -1
    return flatten([(pi + [(li, fi:=fi+1)], n) 
                    for pi, m in arr 
                    for li, n in enumerate(m)], 
                    c+1, max_recurse)

def to_imtx_pmtx(nested_arr, max_recurse=10):
    fmtx = flatten(nested_arr, c=0, max_recurse=max_recurse)
    pmtx = np.array([x[-4:] 
                     for x in fmtx]).T
    imtx = np.array([[z for y in x[:-4] 
                      for z in y] 
                     for x in fmtx]).T
    return imtx, pmtx, fmtx 

def to_poly_4d(poly, ht):
    hmtx = np.array(
        [[1, 0, 0, 0], 
        [0, 1, 0, 0],
        [0, 0, ht, 0], 
        [0, 0, 0, 1]])
    return hmtx @ to_homo(to_homo(np.array(poly).T)) 

blst = [[_hmtx(s["coordinates"][0], s["elevation"]) 
         for s in b["subBuildings"]] 
        for b in data["buildings"][:3]];

pp('len:', len(blst), 
    [len(b) for b in blst], 
    [len(s.T) for b in blst for s in b])

import pandas as pd 
imtx, pmtx, fmtx = to_imtx_pmtx(blst, 4)
pmtx = pmtx#[:, :11]  # 1 bld, 2 sub 
imtx = imtx#[:, :11]

df = pd.DataFrame(
    {"ib": imtx[0], # building idx
     "isl": imtx[1], "isf": imtx[2],  # sub=poly idx
     "ivl": imtx[3], "ivf": imtx[4],  # vector idx 
     "vx": pmtx[0], "vy": pmtx[1], "vz": pmtx[2], "v1": pmtx[3]})

df2poly = lambda x: np.array(x.iloc[:, -4:]).T
[df2poly(x) for i, x in df.groupby(['ib'])][0]  # polys in building 0
#[df2poly(x) for i, x in df.groupby(['isf'])]
#pp(df.shape)
pp("centroid: ", np.mean(df2poly(df), axis=1)[:3].round(2))


